excel_file是包含灯头预测框信息的文件，需要用到里面的image_name、left、right、top、northRotation信息。根据excel中每一行对应的图片名字取出deeplab_folder文件中deeplab好的语义分割图像。然后从预测框往下遍历，找到杆的位置pole_x并根据公式计算其偏航角bearing（包含杆像素最多的列位置），然后还要输出杆底部的位置bottom_x和bottom_y（pole_x这一列中最靠下的包含杆像素的行位置）

In [7]:
import openpyxl
from PIL import Image
import os
from collections import Counter

# 读取Excel文件
excel_file = r"D:\studyarea\area1_data\test_img2.xlsx"  # 你的Excel文件路径
wb = openpyxl.load_workbook(excel_file)
ws = wb.active

# 定义deeplab图片文件夹路径
deeplab_folder = r"D:\studyarea\area1_data\test_img2_deeplab"  # Deeplab分割图片文件夹路径

# 定义颜色范围
color_range = (150, 160)

# 定义列表用于存储结果
results = []

# 找到对应的列索引
header_row = ws[1]
header_to_index = {cell.value: idx for idx, cell in enumerate(header_row)}

# 遍历Excel中的每一行
for row in ws.iter_rows(min_row=2, values_only=True):
    image_name = row[header_to_index["name"]]
    left = row[header_to_index["left"]]
    right = row[header_to_index["right"]]
    top = row[header_to_index["top"]]
    northRotation = row[header_to_index["northRotation"]]
    
    # 打开deeplab图片并加载为PIL Image对象
    image_path = os.path.join(deeplab_folder, image_name)
    image = Image.open(image_path)

    # 创建一个列表用于存储当前列包含灯杆颜色像素的数量
    column_counts = [0] * 2048
    if left-50>0:
        left1 = left-50
    if right+50<2048:
        right1 = right+50
    # 遍历指定区域内的像素，到图片底部
    for y in range(top, image.height):
        for x in range(left1, right1):
            pixel = image.getpixel((x, y))
            # 检查像素颜色是否在指定范围内
            if all(color_range[0] <= pixel[i] <= color_range[1] for i in range(3)):
                column_counts[x] += 1

    # 找到包含灯杆颜色最多的一列
    if all(count == 0 for count in column_counts[left:right]):
      max_x = (left + right) // 2  # 设置为(left + right)的中间值
    else:
      max_x = max(range(len(column_counts)), key=lambda x: column_counts[x]) 
    #print("max_column_id:",max_column_id)
    
    pole_x = max_x
    # 找到包含灯杆颜色最多的一列中包含灯杆颜色且高度最大的像素的 y 坐标
    y = top
    for y in range(top, image.height):
        pixel = image.getpixel((max_x, y))
        # 检查像素颜色是否在指定范围内
        if color_range[0] <= pixel[0] <= color_range[1] and color_range[0] <= pixel[1] <= color_range[1] and color_range[0] <= pixel[2] <= color_range[1]:
                max_y = y
    if(y==image.height):
        max_y = top + 300
    # 将结果存储到列表中
    # 偏航角计算公式（只适用于2048*1024分辨率的图像）
    bearing = ((pole_x * 0.17578125) + 360 - float(northRotation)) % 360
    bottom_x = max_x
    bottom_y = max_y
    results.append((image_name,pole_x,bearing, bottom_x, bottom_y))
# 输出结果
for result in results:
    image_name,pole_x,bearing, bottom_x,bottom_y = result
    print(f"图像名: {image_name}, 包含灯杆颜色最多的一列的绝对列坐标值: {pole_x,bearing}, 最下面像素的坐标: {bottom_x,bottom_y}")


图像名: 000179_22.23813826_114.15422979_202307_9_339.jpg, 包含灯杆颜色最多的一列的绝对列坐标值: (754, 153.5390625), 最下面像素的坐标: (754, 226)
图像名: 000319_22.23822503_114.15425201_202307_9_354.jpg, 包含灯杆颜色最多的一列的绝对列坐标值: (968, 176.15625), 最下面像素的坐标: (968, 226)
图像名: 000551_22.23840056_114.15426771_202307_9_355.jpg, 包含灯杆颜色最多的一列的绝对列坐标值: (248, 48.59375), 最下面像素的坐标: (248, 226)
图像名: 000663_22.23848942_114.15427288_202307_9_5.jpg, 包含灯杆颜色最多的一列的绝对列坐标值: (820, 139.140625), 最下面像素的坐标: (820, 226)
图像名: 000874_22.23696940_114.15354416_202307_8_207.jpg, 包含灯杆颜色最多的一列的绝对列坐标值: (1915, 129.62109375), 最下面像素的坐标: (1915, 535)
图像名: 000983_22.23704640_114.15348601_202307_8_220.jpg, 包含灯杆颜色最多的一列的绝对列坐标值: (1228, 355.859375), 最下面像素的坐标: (1228, 552)
图像名: 001080_22.23711838_114.15342468_202307_8_216.jpg, 包含灯杆颜色最多的一列的绝对列坐标值: (1526, 52.2421875), 最下面像素的坐标: (1526, 628)
图像名: 001300_22.23683473_114.15367225_202205_7_0.jpg, 包含灯杆颜色最多的一列的绝对列坐标值: (472, 82.96875), 最下面像素的坐标: (472, 670)
图像名: 001335_22.23772624_114.15297047_202307_8_220.jpg, 包含灯杆颜色最多的一列的绝对列坐标值: (1822

保存结果到excel_file中

In [4]:
# 读取Excel文件
excel_file = r"D:\studyarea\area1_data\test_img2.xlsx"
wb = openpyxl.load_workbook(excel_file)
ws = wb.active

# 确定新列的起始索引
start_column_index = ws.max_column + 1

# 添加新列标题
ws.cell(row=1, column=start_column_index, value="pole_x")
ws.cell(row=1, column=start_column_index + 1, value="bearing")
ws.cell(row=1, column=start_column_index + 2, value="bottom_x")
ws.cell(row=1, column=start_column_index + 3, value="bottom_y")


# 将结果写入新列
for idx, result in enumerate(results, start=2):
    image_name,pole_x,bearing, bottom_x, bottom_y = result
    ws.cell(row=idx, column=start_column_index, value=pole_x)
    ws.cell(row=idx, column=start_column_index + 1, value=bearing)
    ws.cell(row=idx, column=start_column_index + 2, value=bottom_x)
    ws.cell(row=idx, column=start_column_index + 3, value=bottom_y)

# 保存修改后的Excel文件
wb.save(excel_file)